In [1]:
import networkx as nx
import random

def create_random_regular_graph(n=3000, degree=8):
    """
    Create a random d-regular graph using NetworkX,
    where 'n' is the number of nodes, and 'degree' is the regular degree.
    All edges are assigned weight = 1.
    """
    # Generate a random regular graph
    G = nx.random_regular_graph(d=degree, n=n)

    # Assign weight=1 to every edge
    for (u, v) in G.edges():
        G[u][v]['weight'] = 1

    return G

def randomized_3way_maxcut(G, max_iters=10):
    """
    A simple randomized 3-way Max-Cut algorithm.
    - 'G' is a NetworkX graph with edge weights (defaulting to 1).
    - 'max_iters' is the number of random assignments to try.

    Returns:
        best_cut_value (float or int): the maximum cut value found
        best_partition (dict): a dictionary mapping node -> partition {0,1,2}
    """
    best_cut_value = 0
    best_partition = None
    nodes = list(G.nodes())

    for _ in range(max_iters):
        # Randomly assign each node to one of 3 partitions {0,1,2}
        partition = {node: random.randint(0, 2) for node in nodes}

        # Calculate the cut size
        cut_value = 0
        for (u, v) in G.edges():
            if partition[u] != partition[v]:
                cut_value += G[u][v].get('weight', 1)

        # Update the best cut if this one is better
        if cut_value > best_cut_value:
            best_cut_value = cut_value
            best_partition = partition

    return best_cut_value, best_partition

if __name__ == "__main__":
    # 1. Create the random regular graph
    G = create_random_regular_graph(n=3000, degree=8)

    # 2. Run the randomized 3-way max-cut algorithm (up to 1000 iterations)
    best_value, best_part = randomized_3way_maxcut(G, max_iters=1000)

    print(f"Best cut value found: {best_value}")
    # 'best_part' is a dict node->partition that achieved 'best_value'.
    # You can inspect or process it further as needed.

Best cut value found: 8163


In [4]:
import networkx as nx
import random
import time

def create_random_regular_graph(n, degree=8):
    """
    Creates a random d-regular graph with n nodes using NetworkX.
    Assigns weight=1 to each edge.

    :param n: Number of nodes in the graph
    :param degree: The regular degree (each node has exactly 'degree' neighbors)
    :return: A NetworkX Graph object
    """
    # Generate a random d-regular graph
    G = nx.random_regular_graph(d=degree, n=n)

    # Assign weight=1 to every edge
    for (u, v) in G.edges():
        G[u][v]['weight'] = 1

    return G

def randomized_kway_maxcut(G, k=3, max_iters=1000):
    """
    A simple randomized k-way Max-Cut algorithm.

    :param G: A NetworkX graph with weighted edges (default weight=1).
    :param k: The number of partitions for the k-way Max-Cut.
    :param max_iters: The number of random assignments to try.

    :return: (best_cut_value, best_partition)
             best_cut_value is the maximum cut value found,
             best_partition is a dict node->partition (0..k-1) for that best cut.
    """
    best_cut_value = 0
    best_partition = None
    nodes = list(G.nodes())

    for _ in range(max_iters):
        # Randomly assign each node to one of k partitions
        partition = {node: random.randint(0, k - 1) for node in nodes}

        # Calculate the cut size
        cut_value = 0
        for (u, v) in G.edges():
            # If endpoints differ, edge contributes to the cut
            if partition[u] != partition[v]:
                # Default edge weight is 1
                cut_value += G[u][v].get('weight', 1)

        # Update if we found a better cut
        if cut_value > best_cut_value:
            best_cut_value = cut_value
            best_partition = partition

    return best_cut_value, best_partition

def main():
    # List of node sizes we want to test
    node_sizes = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000]

    # Define the degree (fixed at 8 as requested)
    degree = 8

    # Define the k in k-way Max-Cut
    k = 3  # you can set this to any integer >= 2
    parition_size = [3,4,5,10,100]
    # Define the number of iterations
    max_iters = 10000
    for par in parition_size:
        partition_val = []
        for n in node_sizes:
            # 1. Create the random regular graph of degree 8
            G = create_random_regular_graph(n, degree)

            # 2. Measure time for randomized_kway_maxcut
            start_time = time.time()
            best_cut_value, best_partition = randomized_kway_maxcut(G, k=par, max_iters=max_iters)
            end_time = time.time()

            # Calculate runtime
            runtime = end_time - start_time
            partition_val.append(runtime)

            # print(f"===== n = {n} (k={k}-way) =====")
            # print(f"Best cut value found: {best_cut_value}")
            # print(f"Runtime: {runtime:.2f} seconds\n")
        print("Parition size = {0}".format(par))
        print(partition_val)
        print("-------")

if __name__ == "__main__":
    main()

Parition size = 3
[16.653836965560913, 33.77548313140869, 51.03364896774292, 68.3619430065155, 87.79371190071106, 106.25331497192383, 123.79474091529846, 142.1673698425293, 164.59317803382874, 198.78633379936218, 200.07188296318054, 217.5575602054596, 241.51877307891846]
-------


KeyboardInterrupt: 

In [8]:
import networkx as nx
import random
import time

def create_random_regular_graph(n, degree=8):
    """
    Creates a random d-regular graph with n nodes using NetworkX.
    Assigns weight=1 to each edge.

    :param n: Number of nodes in the graph
    :param degree: The regular degree (each node has exactly 'degree' neighbors)
    :return: A NetworkX Graph object
    """
    # Generate a random d-regular graph
    G = nx.random_regular_graph(d=degree, n=n)

    # Assign weight=1 to every edge
    for (u, v) in G.edges():
        G[u][v]['weight'] = 1

    return G

def randomized_kway_maxcut(G, k=3, max_iters=1000, threshold=0, patience=10):
    """
    A simple randomized k-way Max-Cut algorithm with early stopping.

    :param G: A NetworkX graph with weighted edges (default weight=1).
    :param k: The number of partitions for the k-way Max-Cut.
    :param max_iters: The maximum number of random assignments to try.
    :param threshold: Minimum improvement needed to reset the patience counter.
    :param patience: If no improvement above `threshold` happens for
                     'patience' consecutive iterations, we stop early.

    :return: (best_cut_value, best_partition)
             best_cut_value is the maximum cut value found,
             best_partition is a dict node->partition (0..k-1) for that best cut.
    """
    best_cut_value = 0
    best_partition = None
    nodes = list(G.nodes())

    # Track how many iterations have passed without a sufficient improvement
    iterations_since_improvement = 0

    for i in range(max_iters):
        # Randomly assign each node to one of k partitions
        partition = {node: random.randint(0, k - 1) for node in nodes}

        # Calculate the cut size
        cut_value = 0
        for (u, v) in G.edges():
            if partition[u] != partition[v]:
                cut_value += G[u][v].get('weight', 1)

        # Check improvement
        if cut_value > best_cut_value + threshold:
            best_cut_value = cut_value
            best_partition = partition
            iterations_since_improvement = 0  # reset counter
        else:
            iterations_since_improvement += 1

        # Early stopping check
        if iterations_since_improvement >= patience:
            # Optionally, you can uncomment the print if you want to see early stops
            # print(f"Stopping early at iteration {i} - no improvement > {threshold} for {patience} consecutive iterations.")
            break

    return best_cut_value, best_partition

def main():
    # List of node sizes we want to test
    node_sizes = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000]

    # Define the degree (fixed at 8)
    degree = 8

    # We'll test various k (number of partitions)
    partition_sizes = [3, 4, 5, 10, 100]

    # Define the maximum number of iterations
    max_iters = 10000

    # Early-stopping parameters
    threshold = 50      # minimum improvement in cut value
    patience = 50      # stop if we don't see improvement for 10 iterations

    for par in partition_sizes:
        runtimes_for_each_n = []
        for n in node_sizes:
            # 1. Create the random regular graph of degree 8
            G = create_random_regular_graph(n, degree)

            # 2. Measure time for randomized_kway_maxcut
            start_time = time.time()
            best_cut_value, best_partition = randomized_kway_maxcut(
                G, k=par, max_iters=max_iters, threshold=threshold, patience=patience
            )
            end_time = time.time()

            # Calculate runtime
            runtime = end_time - start_time
            runtimes_for_each_n.append(runtime)

            # (Optional) Print details for debugging or progress
            # print(f"n={n}, k={par}, best_cut_value={best_cut_value}, runtime={runtime:.2f}s")

        print(f"Partition size = {par}")
        print(runtimes_for_each_n)
        print("-------")

if __name__ == "__main__":
    main()

Partition size = 3
[0.09145307540893555, 0.17985892295837402, 0.3018207550048828, 0.39259791374206543, 0.7779510021209717, 0.7849810123443604, 0.7953410148620605, 1.4585978984832764, 1.2604937553405762, 1.2739288806915283, 1.5588369369506836, 2.124526262283325, 2.2678709030151367]
-------
Partition size = 4
[0.10779905319213867, 0.23775887489318848, 0.3010380268096924, 0.7903697490692139, 0.7552390098571777, 0.8173980712890625, 0.7922177314758301, 1.4374911785125732, 1.6115150451660156, 1.0875787734985352, 2.3210060596466064, 2.032412052154541, 1.4824929237365723]
-------
Partition size = 5
[0.09132885932922363, 0.349437952041626, 0.29471898078918457, 0.4068641662597656, 0.8452250957489014, 0.9652531147003174, 1.1437430381774902, 0.8440713882446289, 1.3113317489624023, 1.1861391067504883, 1.2936160564422607, 1.4210898876190186, 2.4729928970336914]
-------
Partition size = 10
[0.09976506233215332, 0.27129125595092773, 0.3759169578552246, 0.4798572063446045, 0.9022870063781738, 0.7244610

In [10]:
import networkx as nx
import random
import time

def create_random_regular_graph(n, degree=8):
    """
    Creates a random d-regular graph with n nodes using NetworkX.
    Assigns weight=1 to each edge.

    :param n: Number of nodes in the graph
    :param degree: The regular degree (each node has exactly 'degree' neighbors)
    :return: A NetworkX Graph object
    """
    # Generate a random d-regular graph
    G = nx.random_regular_graph(d=degree, n=n)

    # Assign weight=1 to every edge
    for (u, v) in G.edges():
        G[u][v]['weight'] = 1

    return G

def randomized_kway_maxcut(G, k=3, max_iters=1000, threshold=0, patience=10):
    """
    A simple randomized k-way Max-Cut algorithm with early stopping.

    :param G: A NetworkX graph with weighted edges (default weight=1).
    :param k: The number of partitions for the k-way Max-Cut.
    :param max_iters: The maximum number of random assignments to try.
    :param threshold: Minimum improvement needed to reset the patience counter.
    :param patience: If no improvement above `threshold` happens for
                     'patience' consecutive iterations, we stop early.

    :return: (best_cut_value, best_partition)
             best_cut_value is the maximum cut value found,
             best_partition is a dict node->partition (0..k-1) for that best cut.
    """
    best_cut_value = 0
    best_partition = None
    nodes = list(G.nodes())

    # Track how many iterations have passed without a sufficient improvement
    iterations_since_improvement = 0

    for i in range(max_iters):
        # Randomly assign each node to one of k partitions
        partition = {node: random.randint(0, k - 1) for node in nodes}

        # Calculate the cut size
        cut_value = 0
        for (u, v) in G.edges():
            if partition[u] != partition[v]:
                cut_value += G[u][v].get('weight', 1)

        # Check improvement
        if cut_value > best_cut_value + threshold:
            best_cut_value = cut_value
            best_partition = partition
            iterations_since_improvement = 0  # reset counter
        else:
            iterations_since_improvement += 1

        # Early stopping check
        if iterations_since_improvement >= patience:
            # Optionally, you can uncomment the print if you want to see early stops
            # print(f"Stopping early at iteration {i} - no improvement > {threshold} for {patience} consecutive iterations.")
            break

    return best_cut_value, best_partition

def main():
    # List of node sizes we want to test
    node_sizes = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000]

    # Define the degree (fixed at 8)
    degree = 8

    # We'll test various k (number of partitions)
    partition_sizes = [3, 4, 5, 10, 100]

    # Define the maximum number of iterations
    max_iters = 10000

    # Early-stopping parameters
    threshold = 1      # minimum improvement in cut value
    patience = 50      # stop if we don't see improvement for 10 iterations

    for par in partition_sizes:
        runtimes_for_each_n = []
        for n in node_sizes:
            # 1. Create the random regular graph of degree 8
            G = create_random_regular_graph(n, degree)

            # 2. Measure time for randomized_kway_maxcut
            start_time = time.time()
            best_cut_value, best_partition = randomized_kway_maxcut(
                G, k=par, max_iters=max_iters, threshold=threshold, patience=patience
            )
            end_time = time.time()

            # Calculate runtime
            runtime = end_time - start_time
            runtimes_for_each_n.append(runtime)

            # (Optional) Print details for debugging or progress
            # print(f"n={n}, k={par}, best_cut_value={best_cut_value}, runtime={runtime:.2f}s")

        print(f"Partition size = {par}")
        print(runtimes_for_each_n)
        print("-------")

if __name__ == "__main__":
    main()

Partition size = 3
[0.09272098541259766, 0.29422593116760254, 0.7135899066925049, 0.7424969673156738, 0.9421460628509521, 0.7536320686340332, 0.9472410678863525, 1.0316870212554932, 2.0556461811065674, 2.0733120441436768, 2.2033779621124268, 1.6323540210723877, 1.422339916229248]
-------
Partition size = 4
[0.12222480773925781, 0.21816110610961914, 0.5013401508331299, 0.5177037715911865, 0.7404510974884033, 0.600783109664917, 1.368124008178711, 1.8271710872650146, 2.5793232917785645, 2.6307950019836426, 1.2778189182281494, 1.7529499530792236, 1.474179983139038]
-------
Partition size = 5
[0.19527411460876465, 0.31960105895996094, 0.48778700828552246, 0.42377710342407227, 1.0189299583435059, 0.7014482021331787, 0.980259895324707, 0.854733943939209, 2.9940409660339355, 1.4128098487854004, 1.4329988956451416, 1.986867904663086, 2.5727181434631348]
-------
Partition size = 10
[0.2392256259918213, 0.22534775733947754, 0.45811891555786133, 0.6428210735321045, 1.1656980514526367, 0.7839391231

In [12]:
Partition size = 3
[0.09272098541259766, 0.29422593116760254, 0.7135899066925049, 0.7424969673156738, 0.9421460628509521, 0.7536320686340332, 0.9472410678863525, 1.0316870212554932, 2.0556461811065674, 2.0733120441436768, 2.2033779621124268, 1.6323540210723877, 1.422339916229248]
-------
Partition size = 4
[0.12222480773925781, 0.21816110610961914, 0.5013401508331299, 0.5177037715911865, 0.7404510974884033, 0.600783109664917, 1.368124008178711, 1.8271710872650146, 2.5793232917785645, 2.6307950019836426, 1.2778189182281494, 1.7529499530792236, 1.474179983139038]
-------
Partition size = 5
[0.19527411460876465, 0.31960105895996094, 0.48778700828552246, 0.42377710342407227, 1.0189299583435059, 0.7014482021331787, 0.980259895324707, 0.854733943939209, 2.9940409660339355, 1.4128098487854004, 1.4329988956451416, 1.986867904663086, 2.5727181434631348]
-------
Partition size = 10
[0.2392256259918213, 0.22534775733947754, 0.45811891555786133, 0.6428210735321045, 1.1656980514526367, 0.7839391231536865, 1.0988500118255615, 1.1000709533691406, 1.3845319747924805, 1.3276150226593018, 2.777590036392212, 2.7067198753356934, 1.8633918762207031]
-------
Partition size = 100
[0.12831401824951172, 0.2657051086425781, 0.396298885345459, 0.765528678894043, 0.7904345989227295, 1.555983066558838, 1.2486999034881592, 1.566702127456665, 1.5053670406341553, 2.531414031982422, 1.518894910812378, 1.3782219886779785, 4.137934923171997]
-------


[5.092720985412598,
 5.2942259311676025,
 5.713589906692505,
 5.742496967315674,
 5.942146062850952,
 5.753632068634033,
 5.9472410678863525,
 6.031687021255493,
 7.055646181106567,
 7.073312044143677,
 7.203377962112427,
 6.632354021072388,
 6.422339916229248]